In [2]:
import datetime as dt
import pandas as pd
import copy
import swifter
import sys

In [22]:
## Mobileinsight rrc csv file.
file = '/home/wmnlab/4/diag_log_qc02_2023-02-04_17-03-23.mi2log_rrc.csv'
mi_rrc_df = pd.read_csv(file)
mi_rrc_df["time"] = mi_rrc_df["time"].swifter.apply(lambda x: pd.to_datetime(x) + dt.timedelta(hours=8))

In [23]:
# file2 = '/home/wmnlab/Code_Test_Space/sheng-ru/2022-12-22/_Bandlock_Udp_B1_B3/sm05/#01/middle/diag_log_sm05_2022-12-22_15-02-55.txt_ml1_new.csv'
file2 = file[:-7] + 'ml1_new.csv'
mi_ml1_df = pd.read_csv(file2)
mi_ml1_df = mi_ml1_df[mi_ml1_df.type_id == 'LTE_PHY_Connected_Mode_Intra_Freq_Meas']
mi_ml1_df["time"] = mi_ml1_df["time"].apply(lambda x: pd.to_datetime(x) + dt.timedelta(hours=8))

# file3 = '/home/wmnlab/Code_Test_Space/sheng-ru/2022-12-22/_Bandlock_Udp_B1_B3/sm05/#01/middle/diag_log_sm05_2022-12-22_15-02-55.txt_nr_ml1_new.csv'
file3 = file[:-7] + 'nr_ml1_new.csv'
mi_nr_ml1_df = pd.read_csv(file3)
mi_nr_ml1_df["time"] = mi_nr_ml1_df["time"].apply(lambda x: pd.to_datetime(x) + dt.timedelta(hours=8))

In [24]:
class REPORTCONFIG:
    def __init__(self, name, parameter):
        self.name = name
        self.parameters = self.parse_parameter(parameter)
    
    def parse_parameter(self, parameter):
        L = []
        start = False
        for i in range(len(parameter)):
            if parameter[i] == "'" and start == False:
                s = ''
                start = True
                continue
            
            if start:
                if parameter[i] == "'":
                    L.append(s)
                    start = False
                s += parameter[i]
        
        P = dict()
        filter = '+-0123456789[]()&'
        for i in range(0,len(L),2):
            x = ''
            for c in L[i+1]:
                if c in filter:
                    x += c
            try:
                P[L[i]] = int(x)
            except:
                P[L[i]] = x
        return P
    def __str__(self):
        return self.name
    def __repr__(self):
        return self.name + ' ' + str(self.parameters)  

class MEASOBJ:
    def __init__(self, obj, freq):
        self.name = obj
        self.freq = freq
    def __str__(self):
        return f'({self.name}, {self.freq})'
    def __repr__(self):
        return f'({self.name}, {self.freq})'

def parse_measIdToAddMod(s):
    a = s.replace('(','')
    a = a.replace(')','')
    a = a.split('&')
    return (a[0], a[1], a[2])

class data_point: # For ml1 data
    def __init__(self, pd_data): ## Input pd_df.iloc[index]
        self.time = pd_data['time']
        self.pci = pd_data['PCI']
        self.servingcell = pd_data['Serving Cell Index']
        self.earfcn = pd_data['EARFCN']
        self.RSRPQ = float(pd_data['RSRP(dBm)']), float(pd_data['RSRQ(dB)'])
        self.neighbor = self.get_nei_pci_signal_strength(pd_data)

    def get_nei_pci_signal_strength(self, pd_data):
        nei_cells = {}
        for i in range(9, len(pd_data), 3):
            if pd_data[i] == '-':
                break
            else:
                nei_cells[pd_data[i]] = (float(pd_data[i+1]), float(pd_data[i+2]))                
        return nei_cells
    def __repr__(self):
        return ' '.join(['PCI:', self.pci,'|(rsrp, rsrq):', str(self.RSRPQ), '|neighbor:', str(self.neighbor), '|Serving Cell:', self.servingcell, '|EARFCN:', self.earfcn])

class nr_data_point:
    def __init__(self, pd_data): ## Input pd_df.iloc[index]
        self.time = pd_data['time']
        # self.servingcell_index = pd_data['Serving Cell Index']
        self.pci = str(pd_data['Serving Cell PCI'])
        self.arfcn = str(pd_data['Raster ARFCN'])
        self.neighbor = self.get_nei_pci_signal_strength(pd_data)

    def get_nei_pci_signal_strength(self, pd_data):
        nei_cells = {}
        for i in range(6, len(pd_data), 3):
            if pd_data[i] == '-':
                break
            else:
                if i == 6:
                    nei_cells[str(pd_data[i])] = (float(pd_data[i+1]), float(pd_data[i+2]))
                else:
                    nei_cells[pd_data[i]] = (float(pd_data[i+1]), float(pd_data[i+2]))                
        return nei_cells
    def __repr__(self):
        return 'PCI: ' + str(self.pci) + ' nei:' + str(self.neighbor)

# RRC + ML1 trying self event part

In [25]:
class CELL:
    def __init__(self, pci=None,rsrp=None, earfcn=None):
        self.pci = pci
        self.rsrp = rsrp
        self.earfcn = earfcn

PCell = CELL()
PSCell = CELL()
SCell_dict = dict()
cell_in_entering_condi = set() # 其實這樣做很有問題，不過再說
cell_in_entering_condi_start_time = {}


def MeasureReport(data): # Self-defined MeasurementReport
    global PCell, cell_in_entering_condi
    if data.servingcell == 'PCell':
        PCell.pci = data.pci
        PCell.rsrp = data.RSRPQ[0]
        PCell.earfcn = data.earfcn
    elif 'SCell' in data.servingcell:
        SCell_dict[data.servingcell] = CELL(pci= data.pci, rsrp=data.RSRPQ[0], earfcn=data.earfcn)


    for id in measId_dict:
        
        Obj_ID = measId_dict[id][0]
        Report_ID = measId_dict[id][1]
        Obj = measobj_dict[Obj_ID]
        Report_Config = report_config_dict[Report_ID]
        
        if Obj.freq != data.earfcn:
            continue

        if Report_Config.name == 'eventA3 (2)':

            off = Report_Config.parameters['off']
            ttt = Report_Config.parameters['ttt']
            hys = Report_Config.parameters['hys']
            
            for nei in list(data.neighbor.keys()):
                nei_rsrp = data.neighbor[nei][0]
                if nei_rsrp - hys >  PCell.rsrp + off: # Entering Condition: Mn + Ofn + Ocn − Hys > Mp + Ofp + Ocp + Off
                    print('### Entering Condition for eventA3:', "PCell:", PCell.pci, PCell.rsrp, "Nei_Cell:", nei, nei_rsrp, mi_ml1_df["time"].iloc[j])
                    if nei not in cell_in_entering_condi:
                        cell_in_entering_condi.add((nei,data.earfcn,ttt))
                        cell_in_entering_condi_start_time[(nei,data.earfcn,ttt)] = data.time
                if nei_rsrp + hys <  PCell.rsrp + off and (nei,data.earfcn,ttt) in cell_in_entering_condi: # Leaving Condition: Mn + Ofn + Ocn + Hys < Mp + Ofp + Ocp + Off
                    print('### Leaving Condition for eventA3:', "PCell:" ,PCell.pci, PCell.rsrp, "Nei_Cell:", nei, nei_rsrp, mi_ml1_df["time"].iloc[j])
                    cell_in_entering_condi.remove((nei,data.earfcn,ttt))
                    cell_in_entering_condi_start_time.pop((nei,data.earfcn,ttt))
        elif Report_Config.name == 'eventA6-r10 (5)':
            Rationable = False
            for key in SCell_dict:
                if SCell_dict[key].earfcn == Obj.freq:
                    SCell = SCell_dict[key]
                    Rationable = True
                    break
            if not Rationable:
                raise 'Weird, no measobject SCell serving?'
        
            off = Report_Config.parameters['off']
            ttt = Report_Config.parameters['ttt']
            hys = Report_Config.parameters['hys']
        
            for nei in list(data.neighbor.keys()):
                nei_rsrp = data.neighbor[nei][0]

                if nei_rsrp - hys >  SCell.rsrp + off: # Entering Condition: Mn + Ocn − Hys > Ms + Ocs + Off
                    print('### Entering Condition for eventA6:', "SCell:", SCell.pci, SCell.rsrp, "Nei_Cell:", nei, nei_rsrp, mi_ml1_df["time"].iloc[j])
                    if nei not in cell_in_entering_condi:
                        cell_in_entering_condi.add((nei,data.earfcn,ttt))
                        cell_in_entering_condi_start_time[(nei,data.earfcn,ttt)] = data.time
                if nei_rsrp + hys <  SCell.rsrp + off and (nei,data.earfcn,ttt) in cell_in_entering_condi: # Laeving Condition: Mn + Ocn + Hys < Ms + Ocs + Off
                    print('### Leaving Condition for eventA6:', "SCell:" ,SCell.pci, SCell.rsrp, "Nei_Cell:", nei, nei_rsrp, mi_ml1_df["time"].iloc[j])
                    cell_in_entering_condi.remove((nei,data.earfcn,ttt))
                    cell_in_entering_condi_start_time.pop((nei,data.earfcn,ttt))
        elif Report_Config.name == 'eventA5 (4)':
            pass
            # thr1 = Report_Config.parameters['thr1']
            # thr2 = Report_Config.parameters['thr2']
            # hys = Report_Config.parameters['hys']
            # ttt = Report_Config.parameters['ttt']

            # for nei in list(data.neighbor.keys()):
            #     nei_rsrp = data.neighbor[nei][0]
            #     if (PCell.rsrp + hys < thr1) and (nei_rsrp - hys > thr2): # Entering Condition1: Mp + Hys < Thresh 1; Entering Condition2: Mn + Ofn + Ocn − Hys > Thresh 2
            #         print('### Entering Condition for eventA5:', "PCell:", PCell.pci, PCell.rsrp, "Nei_Cell:", nei, nei_rsrp, mi_ml1_df["time"].iloc[j])
            #         if nei not in cell_in_entering_condi:
            #             cell_in_entering_condi.add((nei,data.earfcn, ttt))
            #             cell_in_entering_condi_start_time[(nei,data.earfcn, ttt)] = data.time
            #     if ((PCell.rsrp-hys > thr1) or (nei_rsrp + hys < thr2)) and (nei,data.earfcn, ttt) in cell_in_entering_condi: # Leaving Condition1: Mp − Hys > Thresh 1; Leaving Condition2: Mn + Ofn + Ocn + Hys < Thresh 2
            #         print('### Leaving Condition for eventA5:', "PCell:" ,PCell.pci, PCell.rsrp, "Nei_Cell:", nei, nei_rsrp, mi_ml1_df["time"].iloc[j])
            #         cell_in_entering_condi.remove((nei,data.earfcn, ttt))
            #         cell_in_entering_condi_start_time.pop((nei,data.earfcn, ttt))
        elif Report_Config.name == 'eventA2 (1)': # A2 還沒debug過
            thr = Report_Config.parameters['thr']
            hys = Report_Config.parameters['hys']
            ttt = Report_Config.parameters['ttt']

            if PCell.rsrp + hys < thr:  # Entering Condition: Ms + Hys < Thresh
                print('### Entering Condition for eventA2:', "PCell:", PCell.pci, PCell.rsrp)
                if nei not in cell_in_entering_condi:
                    cell_in_entering_condi.add((PCell.pci, PCell.earfcn, ttt))
                    cell_in_entering_condi_start_time[(PCell.pci, PCell.earfcn, ttt)] = data.time
            if PCell.rsrp - hys > thr and (PCell.pci, PCell.earfcn, ttt) in cell_in_entering_condi: # Leaving Condition: Ms − Hys > Thresh
                print('### Leaving Condition for eventA2:', "PCell:" ,PCell.pci, PCell.rsrp, "Nei_Cell:", nei, nei_rsrp, mi_ml1_df["time"].iloc[j])
                cell_in_entering_condi.remove((PCell.pci, PCell.earfcn, ttt))
                cell_in_entering_condi_start_time.pop((PCell.pci, PCell.earfcn, ttt))
        else:
            pass


def nr_MeasureReport(data): # Self-defined MeasurementReport; deal with lte B1 event and NR A2, A3 event
    global cell_in_entering_condi

    for id in measId_dict: # 特別處裡B1事件
        Obj_ID = measId_dict[id][0]
        Report_ID = measId_dict[id][1]
        Obj = measobj_dict[Obj_ID]
        Report_Config = report_config_dict[Report_ID]

        if Obj.freq != data.arfcn:
            continue

        if Report_Config.name == 'eventB1-NR-r15 (5)':
            thr = Report_Config.parameters['thr'].split('&')
            thr[0], thr[1] = int(thr[0][1:]), int(thr[1][:-1])
            hys = Report_Config.parameters['hys']
            ttt = Report_Config.parameters['ttt']
 
            for nei in list(data.neighbor.keys()):
                nei_rsrp = data.neighbor[nei][0]
                if nei_rsrp - hys > thr[1]: # Entering Condition1: Mn + Ofn − Hys > Thresh
                    print('### Entering Condition for eventB1:', "Nei_Cell:", nei, nei_rsrp, mi_nr_ml1_df["time"].iloc[k])
                    if nei not in cell_in_entering_condi:
                        cell_in_entering_condi.add((nei, data.arfcn, ttt))
                        cell_in_entering_condi_start_time[(nei, data.arfcn, ttt)] = data.time

                if nei_rsrp + hys < thr[0] and (nei, data.arfcn, ttt) in cell_in_entering_condi: # Leaving Condition1: Mn + Ofn + Hys < Thresh
                    print('### Leaving Condition for eventB1:', "Nei_Cell:", nei, nei_rsrp, mi_nr_ml1_df["time"].iloc[k])
                    cell_in_entering_condi.remove((nei, data.arfcn, ttt))
                    cell_in_entering_condi_start_time.pop((nei, data.arfcn, ttt))
        else:
            continue
    
    global PSCell
    
    if data.pci != '65535':
        PSCell.pci = data.pci
        PSCell.rsrp = data.neighbor[data.pci][0]
        PSCell.earfcn = data.arfcn
    else:
        return
    for id in nr_measId_dict: # NR gNB events!
        Obj_ID = nr_measId_dict[id][0]
        Report_ID = nr_measId_dict[id][1]
        Obj = nr_measobj_dict[Obj_ID]
        Report_Config = nr_report_config_dict[Report_ID]

        if Obj.freq != data.arfcn:
            continue

        if Report_Config.name == 'eventA3 (2)':
            off = Report_Config.parameters['off']
            hys = Report_Config.parameters['hys']
            ttt = Report_Config.parameters['ttt']
 
            for nei in list(data.neighbor.keys()):
                if nei == PSCell.pci:
                    continue
                nei_rsrp = data.neighbor[nei][0]
                if nei_rsrp - hys > PSCell.rsrp + off: # Entering Condition1: Mn + Ofn + Ocn – Hys > Mp + Ofp + Ocp + Off
                    print('### Entering Condition for nr-eventA3:', "PSCell: ", PSCell.pci,PSCell.rsrp, "Nei_Cell:", nei, nei_rsrp, mi_nr_ml1_df["time"].iloc[k])
                    if nei not in cell_in_entering_condi:
                        cell_in_entering_condi.add((nei, data.arfcn, ttt))
                        cell_in_entering_condi_start_time[(nei, data.arfcn, ttt)] = data.time
                if nei_rsrp + hys < PSCell.rsrp + off and (nei, data.arfcn, ttt) in cell_in_entering_condi: # Leaving Condition1: Mn + Ofn + Ocn + Hys < Mp + Ofp + Ocp + Off
                    print('### Leaving Condition for nr-eventA3:', "PSCell:", PSCell.pci, PSCell.rsrp, "Nei_Cell:", nei, nei_rsrp, mi_nr_ml1_df["time"].iloc[k])
                    cell_in_entering_condi.remove((nei, data.arfcn, ttt))
                    cell_in_entering_condi_start_time.pop((nei, data.arfcn, ttt))
        else:
            continue
import copy

def clean_cell_in_entering_config(time):
    global cell_in_entering_condi, cell_in_entering_condi_start_time
    c = copy.deepcopy(cell_in_entering_condi_start_time)
    for element in cell_in_entering_condi_start_time:
        if time-cell_in_entering_condi_start_time[element] > dt.timedelta(microseconds=element[2]):
            cell_in_entering_condi.remove(element)
            c.pop(element)
    cell_in_entering_condi_start_time = copy.deepcopy(c)

In [26]:
measobj_dict = {}
report_config_dict = {}
measId_dict = {}

nr_measobj_dict = {}
nr_report_config_dict = {}
nr_measId_dict = {}

# Output file here!
f = open("/home/wmnlab/record.txt", "w")
original_stdout = sys.stdout
sys.stdout = f

event_list = [
    "lte-measurementReport", "lte-MeasObjectToAddMod", "lte-ReportConfigToAddMod", "lte-measIdToRemoveList","lte-MeasIdToAddMod", 
    "lte-rrc.t304","rrcConnectionRelease", "rrcConnectionSetup","rrcConnectionReestablishmentRequest","rrcConnectionReestablishmentReject",
    "SCellToAddMod-r10", "sCellToReleaseList-r10", "nr-rrc.t304",
    "nr-MeasObjectToAddMod", "nr-ReportConfigToAddMod", "nr-MeasIdToAddMod", "nr-measurementReport","scgFailureInformationNR-r15",
    "nr-Config-r15: release (0)", 
    ]

RRC_connected = True

i = 0
j = 0
k = 0
while i < len(mi_rrc_df)-1 or j < len(mi_ml1_df)-1 or k < len(mi_nr_ml1_df)-1:
    if len(mi_nr_ml1_df) == 0:
        order = {'rrc': mi_rrc_df['time'].iloc[i], 'ml1':mi_ml1_df['time'].iloc[j]}
    else:
        order = {'rrc': mi_rrc_df['time'].iloc[i], 'ml1':mi_ml1_df['time'].iloc[j], 'nr_ml1': mi_nr_ml1_df['time'].iloc[k]}
    if i == len(mi_rrc_df)-1:
        order.pop('rrc')
    if j == len(mi_ml1_df)-1:
        order.pop('ml1')
    if k == len(mi_nr_ml1_df) -1 and len(mi_nr_ml1_df) != 0:
        order.pop('nr_ml1')
    GO = min(order, key=order.get)
    
    if GO == 'nr_ml1':
        D = nr_data_point(mi_nr_ml1_df.iloc[k])
        # clean_cell_in_entering_config(D.time)
        # nr_MeasureReport(D)
        k += 1
    elif GO == 'ml1':
        D = data_point(mi_ml1_df.iloc[j])
        # clean_cell_in_entering_config(D.time)
        # MeasureReport(D)
        j += 1
    elif GO == 'rrc':
        # clean_cell_in_entering_config(mi_rrc_df["time"].iloc[i])
        if mi_rrc_df['type_id'].iloc[i] == "5G_NR_RRC_OTA_Packet" or mi_rrc_df['type_id'].iloc[i] == "LTE_RRC_Serv_Cell_Info":
            i += 1
            continue

        for event in event_list:
            if mi_rrc_df[event].iloc[i] != 0 and mi_rrc_df[event].iloc[i] != '0':
                if event == "SCellToAddMod-r10":
                    if mi_rrc_df["physCellId-r10"].iloc[i] != "nr or cqi report":
                        print('================================================')
                        print(mi_rrc_df['time'].iloc[i])
                    break
                else:
                    print('================================================')
                    print(mi_rrc_df['time'].iloc[i])
                    break

        if mi_rrc_df["rrcConnectionRelease"].iloc[i] == 1:
            print("Release", "PCI:",mi_rrc_df["PCI"].iloc[i], int(mi_rrc_df["Freq"].iloc[i]))
            RRC_connected = False
            print(f"RRC_connected: {RRC_connected}")
            measobj_dict, report_config_dict, measId_dict = {}, {}, {}
            nr_measobj_dict, nr_report_config_dict, nr_measId_dict = {}, {}, {}
            cell_in_entering_condi.clear()

        if mi_rrc_df["rrcConnectionSetup"].iloc[i] == 1:
            print("Setup","->", mi_rrc_df["PCI"].iloc[i], int(mi_rrc_df["Freq"].iloc[i]))
            RRC_connected = True
            print(f"RRC_connected: {RRC_connected}")           
        
        if not RRC_connected:
            i+=1
            continue
        
        if i == 0:
            print("Initial", "PCI:",mi_rrc_df["PCI"].iloc[0], int(mi_rrc_df["Freq"].iloc[0]))

        if mi_rrc_df["lte-rrc.t304"].iloc[i] == 1:
            pci = mi_rrc_df["PCI"].iloc[i]
            target = int(mi_rrc_df["lte_targetPhysCellId"].iloc[i])
            Freq = int(mi_rrc_df["Freq"].iloc[i])
            dl_CarrierFreq = int(mi_rrc_df["dl-CarrierFreq"].iloc[i])
            print('HO', pci, Freq, '->', target, dl_CarrierFreq)

        if mi_rrc_df["lte-measurementReport"].iloc[i] == 1:
            print("LTE-measurementReport")
            id = str(int(mi_rrc_df['measId'].iloc[i]))
            try:
                x = measId_dict[id]
                event = report_config_dict[x[1]].name
                if 'B1' in event:
                    print(id, measId_dict[id], measobj_dict[x[0]], report_config_dict[x[1]], mi_rrc_df['pci-r15'].iloc[i])
                else :
                    print(id, measId_dict[id], measobj_dict[x[0]], report_config_dict[x[1]], mi_rrc_df['physCellId'].iloc[i])
                
                print('parameters:', report_config_dict[x[1]].parameters)
            except:
                print(id, "Unknown")

        if mi_rrc_df["nr-measurementReport"].iloc[i] == 1:
            print("NR-measurementReport")
            id = str(int(mi_rrc_df['measId'].iloc[i]))
            try:
                x = nr_measId_dict[id]
                print(id, nr_measId_dict[id], nr_measobj_dict[x[0]], nr_report_config_dict[x[1]], mi_rrc_df['physCellId.1'].iloc[i])
                print('parameters:', nr_report_config_dict[x[1]].parameters)
            except:
                print(id, "Unknown")   

        if mi_rrc_df["lte-MeasObjectToAddMod"].iloc[i] == 1:
            print("MeasObjectToAddMod")
            Id_list = str(mi_rrc_df["measObjectId"].iloc[i]).split('@')
            measobj_list = mi_rrc_df["measObject"].iloc[i].split('@')
            carrierFreq_list = str(mi_rrc_df["carrierFreq"].iloc[i]).split('@')
            carrierFreq_r15_list = str(mi_rrc_df["carrierFreq-r15"].iloc[i]).split('@')
            for a in range(len(Id_list)):
                if measobj_list[a] == "measObjectEUTRA (0)":
                    measobj_dict[Id_list[a]] = MEASOBJ(measobj_list[a], carrierFreq_list[0])
                    carrierFreq_list.pop(0)
                elif measobj_list[a] == "measObjectNR-r15 (5)":
                    measobj_dict[Id_list[a]] = MEASOBJ(measobj_list[a], carrierFreq_r15_list[0])
                    carrierFreq_r15_list.pop(0)
            print(measobj_dict)   

        if mi_rrc_df["nr-MeasObjectToAddMod"].iloc[i] == 1:
            print("nr-MeasObjectToAddMod")
            Id_list = mi_rrc_df["measObjectId"].iloc[i].split('@')
            measobj_list = mi_rrc_df["measObject"].iloc[i].split('@')
            ssbFrequency_list = str(int(mi_rrc_df["ssbFrequency"].iloc[i])).split('@')

            for a in range(len(Id_list)):
                if measobj_list[a] == "measObjectNR (0)":
                    nr_measobj_dict[Id_list[a]] = MEASOBJ(measobj_list[a], int(ssbFrequency_list[0]))
                    ssbFrequency_list.pop(0)     
            print(nr_measobj_dict)
            
        if mi_rrc_df["lte-ReportConfigToAddMod"].iloc[i] == 1:
            print("ReportConfigToAddMod")
            reportConfigId_list = str(mi_rrc_df["lte-reportConfigId"].iloc[i]).split('@')
            eventId_list = mi_rrc_df["lte-eventId"].iloc[i].split('@')
            parameter_list = mi_rrc_df["lte-parameter"].iloc[i].split('@')
            for a in range(len(reportConfigId_list)):
                report_config_dict[reportConfigId_list[a]] = REPORTCONFIG(eventId_list[a], parameter_list[a])
            print(report_config_dict)

        if mi_rrc_df["nr-ReportConfigToAddMod"].iloc[i] == 1: #############
            print("nr-ReportConfigToAddMod")
            reportConfigId_list = mi_rrc_df["nr-reportConfigId"].iloc[i].split('@')
            eventId_list = mi_rrc_df["nr-eventId"].iloc[i].split('@')
            parameter_list = mi_rrc_df["nr-parameter"].iloc[i].split('@')
            for a in range(len(reportConfigId_list)):
                nr_report_config_dict[reportConfigId_list[a]] = REPORTCONFIG(eventId_list[a], parameter_list[a])
            print(nr_report_config_dict)

        if mi_rrc_df["lte-measIdToRemoveList"].iloc[i] != '0':
            print("measIdToRemoveList")
            measIdToRemove_list = str(mi_rrc_df["lte-measIdToRemoveList"].iloc[i]).split('@')
            if len(measIdToRemove_list) == 32:
                measId_dict = {}
            else:
                for a in range(len(measIdToRemove_list)):
                    measId_dict.pop(measIdToRemove_list[a])
            print(measId_dict)
        if mi_rrc_df["lte-MeasIdToAddMod"].iloc[i] != '0':
            print("MeasIdToAddMod")
            MeasIdToAdd_list = mi_rrc_df["lte-MeasIdToAddMod"].iloc[i].split('@')
            for a in range(len(MeasIdToAdd_list)):
                x = parse_measIdToAddMod(MeasIdToAdd_list[a])
                measId_dict[x[0]] = (x[1],x[2])
            print(measId_dict)
        if mi_rrc_df["nr-MeasIdToAddMod"].iloc[i] != '0' and mi_rrc_df["nr-MeasIdToAddMod"].iloc[i] != 0:
            print("nr-MeasIdToAddMod")
            MeasIdToAdd_list = mi_rrc_df["nr-MeasIdToAddMod"].iloc[i].split('@')
            for a in range(len(MeasIdToAdd_list)):
                x = parse_measIdToAddMod(MeasIdToAdd_list[a])
                nr_measId_dict[x[0]] = (x[1],x[2])
            print(nr_measId_dict)
        if mi_rrc_df["rrcConnectionReestablishmentRequest"].iloc[i] == 1:
            target = mi_rrc_df["physCellId.3"].iloc[i]
            print(f"rrcConnectionReestablishmentRequest to PCI:{target}")
            measobj_dict, report_config_dict, measId_dict = {}, {}, {}
            nr_measobj_dict, nr_report_config_dict, nr_measId_dict = {}, {}, {}
        if mi_rrc_df["rrcConnectionReestablishmentComplete"].iloc[i] == 1:
            pci = mi_rrc_df["PCI"].iloc[i]
            print(f"successful reestablish to {pci}")
        if mi_rrc_df["rrcConnectionReestablishmentReject"].iloc[i] == 1:
            print(f"rrcConnectionReestablishmentReject")
        if mi_rrc_df["sCellToReleaseList-r10"].iloc[i] != '0' and mi_rrc_df["sCellToReleaseList-r10"].iloc[i] != 0:
            ind = str(mi_rrc_df["SCellIndex-r10"].iloc[i])
            print("Release SCell ID:", ind.split('@'))
        if mi_rrc_df["SCellToAddMod-r10"].iloc[i] == 1 and mi_rrc_df["physCellId-r10"].iloc[i] != "nr or cqi report":
            ind = str(mi_rrc_df["SCellIndex-r10.1"].iloc[i]).split('@')
            pci = mi_rrc_df["physCellId-r10"].iloc[i].split('@')
            dlfreq = str(mi_rrc_df["dl-CarrierFreq-r10"].iloc[i]).split('@')
            S = [(ind[a], pci[a], dlfreq[a]) for a in range(len(ind))]
            for s in S:
                print('Add Scell:', s)
        if mi_rrc_df["nr-rrc.t304"].iloc[i] == 1:
            pci = int(mi_rrc_df["nr_physCellId"].iloc[i])
            if mi_rrc_df["SCellToAddMod-r10"].iloc[i] == 1: 
                print(f'**NR-setup to pci: {pci}.**')
            else:
                print(f'**NR-Ho to pci: {pci}.**')

        if mi_rrc_df["scgFailureInformationNR-r15"].iloc[i]:
            print("NR-SCG Failure.")
            nr_measobj_dict, nr_report_config_dict, nr_measId_dict = {}, {}, {}
        
        if mi_rrc_df["nr-Config-r15: release (0)"].iloc[i]:
            print("NR PSCell release.")
            nr_measobj_dict, nr_report_config_dict, nr_measId_dict = {}, {}, {}
        
        i += 1

sys.stdout = original_stdout
f.close()
print(f'Finished')

# Only RRC part

In [ ]:
# measobj_dict = {}
# report_config_dict = {}
# measId_dict = {}

# event_list = [
#     "measurementReport", "MeasObjectToAddMod", "ReportConfigToAddMod", "measIdToRemoveList","MeasIdToAddMod", 
#     "lte-rrc.t304","rrcConnectionRelease", "rrcConnectionSetup",
#     ]
# RRC_connected = True
# for i in range(len(mi_rrc_df)):
#     for event in event_list:
#         if mi_rrc_df[event].iloc[i] != 0 and mi_rrc_df[event].iloc[i] != '0':
#             print('================================================')
#             print(mi_rrc_df['time'].iloc[i])
#             break
#     if mi_rrc_df["rrcConnectionRelease"].iloc[i] == 1:
#         print("Release", "PCI:",mi_rrc_df["PCI"].iloc[i], mi_rrc_df["Freq"].iloc[i])
#         RRC_connected = False
#         print(f"RRC_connected: {RRC_connected}")
#     if mi_rrc_df["rrcConnectionSetup"].iloc[i] == 1:
#         print("Setup","->",mi_rrc_df["PCI"].iloc[i], mi_rrc_df["Freq"].iloc[i])
#         RRC_connected = True
#         print(f"RRC_connected: {RRC_connected}")
#     if not RRC_connected:
#         continue
    
#     if i == 0:
#         print("Initial", "PCI:",mi_rrc_df["PCI"].iloc[0], mi_rrc_df["Freq"].iloc[0])
#     if mi_rrc_df["lte-rrc.t304"].iloc[i] == 1:
#         pci = mi_rrc_df["PCI"].iloc[i]
#         target = mi_rrc_df["lte_targetPhysCellId"].iloc[i]
#         Freq = mi_rrc_df["Freq"].iloc[i]
#         dl_CarrierFreq = mi_rrc_df["dl-CarrierFreq"].iloc[i]
#         print('HO', pci, Freq, '->', target, dl_CarrierFreq)
#     if mi_rrc_df["measurementReport"].iloc[i] == 1:
#         print("measurementReport")
#         id = str(mi_rrc_df['measId'].iloc[i])
#         if id == "none":
#             print(id)
#             continue
#         try:
#             x = measId_dict[id]
#             print(id, measId_dict[id], measobj_dict[x[0]], report_config_dict[x[1]], mi_rrc_df['physCellId'].iloc[i])
#             print('parameters:', report_config_dict[x[1]].parameters)
#         except:
#             print(id, "Unknown")
#     if mi_rrc_df["MeasObjectToAddMod"].iloc[i] == 1:
#         print("MeasObjectToAddMod")
#         Id_list = mi_rrc_df["measObjectId"].iloc[i].split('@')
#         measobj_list = mi_rrc_df["measObject"].iloc[i].split('@')
#         carrierFreq_list = mi_rrc_df["carrierFreq"].iloc[i].split('@')
#         carrierFreq_r15_list = str(mi_rrc_df["carrierFreq-r15"].iloc[i]).split('@')
#         for j in range(len(Id_list)):
#             if measobj_list[j] == "measObjectEUTRA (0)":
#                 measobj_dict[Id_list[j]] = MEASOBJ(measobj_list[j], carrierFreq_list[0])
#                 carrierFreq_list.pop(0)
#             elif measobj_list[j] == "measObjectNR-r15 (5)":
#                 measobj_dict[Id_list[j]] = MEASOBJ(measobj_list[j], carrierFreq_r15_list[0])
#                 carrierFreq_r15_list.pop(0)
#         print(measobj_dict)
#     if mi_rrc_df["ReportConfigToAddMod"].iloc[i] == 1:
#         print("ReportConfigToAddMod")
#         reportConfigId_list = mi_rrc_df["reportConfigId"].iloc[i].split('@')
#         eventId_list = mi_rrc_df["eventId"].iloc[i].split('@')
#         parameter_list = mi_rrc_df["parameter"].iloc[i].split('@')
#         for j in range(len(reportConfigId_list)):
#             report_config_dict[reportConfigId_list[j]] = REPORTCONFIG(eventId_list[j], parameter_list[j])
#         print(report_config_dict)
#     if mi_rrc_df["measIdToRemoveList"].iloc[i] != '0':
#         print("measIdToRemoveList")
#         measIdToRemove_list = mi_rrc_df["measIdToRemoveList"].iloc[i].split('@')
#         if len(measIdToRemove_list) == 32:
#             measId_dict = {}
#         else:
#             for j in range(len(measIdToRemove_list)):
#                 measId_dict.pop(measIdToRemove_list[j])
#         print(measId_dict)
#     if mi_rrc_df["MeasIdToAddMod"].iloc[i] != '0':
#         print("MeasIdToAddMod")
#         MeasIdToAdd_list = mi_rrc_df["MeasIdToAddMod"].iloc[i].split('@')
#         for j in range(len(MeasIdToAdd_list)):
#             x = parse_measIdToAddMod(MeasIdToAdd_list[j])
#             measId_dict[x[0]] = (x[1],x[2])
#         print(measId_dict)